# MATH2504 S2, 2024, Project 1 Submission

### Trisztan Harai, 47483073

<html>
    <a href="https://github.com/TooMuchWater78/Trisztan-Harai-2504-2024-PROJECT1.git">
    GitHub repo
    </a>
<html>

## Task 1

## Task 2

When changing `max_degree_allowed` from `400` to `100` an error occurs when calling `prod_test_poly`. Looking at the code, we see that within this function, the error occurs in the second `for` loop section:

In [ ]:
for _ in 1:N
    p_base = Polynomial(Term(1,0))
    for _ in 1:N_prods
        p = rand(Polynomial)
        prod = p_base*p
        @assert leading(prod) == leading(p_base)*leading(p)
        p_base = prod
    end
end

The relevant input values are `N = 1000` and `N_prods = 20`. Looking at `rand(::Polynomial)`, we see that the vast majority of randomly generated polynomials will have degree between 0 and 10. Thus, with `N_prods = 20`, the maximum degree of the polynomial generated in the inner for loop via multiplication will be about 200. This is less than the original `max_degree_allowed` of `400`, but larger than the new `100`. Therefore, the created polynomial has degree too high and an error is thrown.

Since `max_degree_allowed` had no integral role in any of the defined functions beyond acting as a 'limiter' on size, it's removal causes no problems. Terms, polynomials, etc. are still perfectly well defined without `max_degree_allowed`. The change was made in commit 26 in the repo, and the files `poly_factorization_project.jl`, `polynomial.jl` and `term.jl` were edited.

## Task 3

The below code implements `lowest_to_highest` and all pretty printing, apart from the unicode superscripts:

In [ ]:
if (@isdefined lowest_to_highest) && lowest_to_highest == true
            for (i,t) in enumerate(p.terms)
                if !iszero(t)
                    print(io, i == 1 ? t : (t.coeff < 0 ? " - $(string(t)[2:end])" : " + $t"))  # if coefficient is negative, print minus sign
                end
            end
        else
            for (i,t) in enumerate(reverse(p.terms))  # if lowest_to_highest is false, print polynomial in descending order
                if !iszero(t)
                    print(io, i == 1 ? t : (t.coeff < 0 ? " - $(string(t)[2:end])" : " + $t"))  # if coefficient is negative, print minus sign
                end
            end
        end

Since the code has to run even if `lowest_to_highest` doesn't exist, I use an `@isdefined` call to avoid a crash in case it isn't defined. Then if `@isdefined` returns `true`, I check the actual value of `lowest_to_highest`. If it is true, I leave the enumeration of the terms in the original order; if it's false, I reverse the order, thus printing the polynomial with terms in descending order.

To achieve pretty printing, I use the below code, checking whether the term's coefficient is negative, whether the degree is 1, whether the coefficient is `1` or `-1` and if the term being printed is the first in the polynomial.

In [ ]:
"""
Print a number in unicode superscript.
"""
function number_superscript(i::Int)
    if i < 0
        c = [Char(0x207B)]  # superscript minus sign
    else
        c = []
    end

    # digits separates the digits of i into an array in reverse order (right to left); this order must be reversed for correct printing
    for j in reverse(digits(abs(i)))
        # 1, 2 and 3 do not follow the same unicode pattern as 4 onwards
        if j == 0
            push!(c, Char(0x2070))
        elseif j == 1
            push!(c, Char(0x00B9))
        elseif j == 2
            push!(c, Char(0x00B2))
        elseif j == 3
            push!(c, Char(0x00B3))
        else
            push!(c, Char(0x2070+j))
        end
    end
    return join(c)
end

"""
Show a term.
"""
function show(io::IO, t::T) where T <: AbsTerm
    t.degree == 0 && return print(io, "$(t.coeff)")  # do not print x for constant terms
    if abs(t.coeff) == 1  # do not print coefficient 1 explicitly
        t.degree == 1 ? print(io, "x") : print(io, "x$(number_superscript(t.degree))")
    else # do not print exponent if degree is 1; otherwise print exponent with unicode superscript
        t.degree == 1 ? print(io, "$(t.coeff)⋅x") : print(io, "$(t.coeff)⋅x$(number_superscript(t.degree))")
    end
end

The above code takes care of the unicode superscript printing. As mentioned in the comments, `digits` produces an array filled with the digits of `i` in reverse order (i.e. right to left); to print properly, this order must be reversed. The `elseif` cascade is necessary, as the unicode values of `1, 2, 3` as superscripts do not align with the general pattern from `4` onwards.

I have explained in the comments of the `show` function what each part of the code does.

## Task 4

To make the code cleaner, I introduced abstract types called `AbsPoly` and `AbsTerm`, under which `Polynomial` and `PolynomialBig`, `Term` and `TermBig`, respectively, became subtypes. This way, all the original functions made by Yoni also work for `PolynomialBig` and `TermBig`. The definitions of `PolynomialBig` and `TermBig` are shown below.

In [ ]:
"""
A TermBig.
"""
struct TermBig <: AbsTerm  #structs are immutable by default
    coeff::BigInt
    degree::Int
    function TermBig(coeff::Integer, degree::Int)
        degree < 0 && error("Degree must be non-negative")
        coeff != 0 ? new(big(coeff),degree) : new(big(coeff),0)
    end
end

"""
Define equality for TermBig
"""
==(a::TermBig, b::TermBig) = (a.coeff == b.coeff) && (a.degree == b.degree)

"""
A PolynomialBig type - designed to be for polynomials with integer coefficients.
"""
struct PolynomialBig <: AbsPoly

    #A zero packed vector of terms
    #Terms are assumed to be in order
    #until the degree of the polynomial. The leading term (i.e. last) is assumed to be non-zero except 
    #for the zero polynomial where the vector is of length 1.
    #Note: at positions where the coefficient is 0, the power of the term is also 0 (this is how the TermBig type is designed)
    #The maximum length allowed for the vector is max_degree+1
    terms::Vector{TermBig}   
    
    #Inner constructor of 0 polynomial
    PolynomialBig() = new([zero(TermBig)])

    #Inner constructor of polynomial based on arbitrary list of terms
    function PolynomialBig(vt::Vector{TermBig})

        #Filter the vector so that there is not more than a single zero term
        vt = filter((t)->!iszero(t), vt)
        if isempty(vt)
            vt = [zero(TermBig)]
        end

        max_degree = maximum((t)->t.degree, vt)
        terms = [zero(TermBig) for i in 0:max_degree] #First set all terms with zeros

        #now update based on the input terms
        for t in vt
            terms[t.degree + 1] = t #+1 accounts for 1-indexing
        end
        return new(terms)
    end
end

Equality had to be redefined for `TermBig` for technical reasons, as it uses `BigInt` rather than a "regular" `Int` type. An example of a test for `Polynomial`, and the corresponding duplicate for `PolynomialBig`, are shown below. 

In [ ]:
"""
Executes all polynomial factorization tests in this file.
"""
function factorization_tests()
    factor_test_poly()
    factor_test_polyBig()
end

"""
Test factorization of polynomials.
"""
function factor_test_poly(;N::Int = 10, seed::Int = 0, primes::Vector{Int} = [5,17,19])
    Random.seed!(seed)
    for prime in primes
        print("\ndoing prime = $prime \t")
        for _ in 1:N
            print(".")
            p = rand(Polynomial)
            factorization = factor(p, prime)
            pr = mod(expand_factorization(factorization),prime)
            @assert mod(p-pr,prime) == 0 
        end
    end

    println("\nfactor_test_poly - PASSED")
end

"""
Test factorization of BigInt polynomials.
"""
function factor_test_polyBig(;N::Int = 10, seed::Int = 0, primes::Vector{Int} = [5,17,19])
    Random.seed!(seed)
    for prime in primes
        print("\ndoing prime = $prime \t")
        for _ in 1:N
            print(".")
            p = rand(PolynomialBig)
            factorization = factor(p, prime)
            pr = mod(expand_factorization(factorization),prime)
            @assert mod(p-pr,prime) == 0 
        end
    end

    println("\nfactor_test_polyBig - PASSED")
end

As per the instructions, overflow tests were also written to demonstrate the main advantage of using `BigInt` coefficients.

In [ ]:
function polynomial_overflow_tests()
    poly_overflow()
    polyBig_overflow()
end

"""
Tests whether a polynomial can overflow (note: should fail).
"""
function poly_overflow(; N::Int = 128)
    p = x_poly(Polynomial)
    for _ in 1:N
        @assert leading(p*2) > leading(p)
        p *= 2
    end
    println("poly_overflow - PASSED")
end

"""
Tests whether a BigInt polynomial can overflow.
"""
function polyBig_overflow(; N::Int = 128)
    p = x_poly(PolynomialBig)
    for _ in 1:N
        @assert leading(p*2) > leading(p)
        p *= 2
    end
    println("polyBig_overflow - PASSED")
end

One can see that the `poly_overflow` test is guaranteed to fail (assert `false`) for the `Polynomial` type. It passes for `PolynomialBig`. Below is the code used to empirically compare run-times for `Polynomial` and `PolynomialBig` multiplication. Output is given in the subsequent cell.

In [ ]:
"""
Executes all polynomial product tests in this file.
"""
function polynomial_product_tests()
    @time prod_test_poly()
    @time prod_test_polyBig()
end

"""
Test product of polynomials.
"""
function prod_test_poly(;N::Int = 100, N_prods::Int = 10, seed::Int = 0)
    Random.seed!(seed)
    for _ in 1:N
        p1 = rand(Polynomial)
        p2 = rand(Polynomial)
        prod = p1*p2
        @assert leading(prod) == leading(p1)*leading(p2)
    end

    for _ in 1:N
        p_base = Polynomial(Term(1,0))
        for _ in 1:N_prods
            p = rand(Polynomial)
            prod = p_base*p
            @assert leading(prod) == leading(p_base)*leading(p)
            p_base = prod
        end
    end
    println("prod_test_poly - PASSED")
end

"""
Test product of BigInt polynomials.
"""
function prod_test_polyBig(;N::Int = 100, N_prods::Int = 10, seed::Int = 0)
    Random.seed!(seed)
    for _ in 1:N
        p1 = rand(PolynomialBig)
        p2 = rand(PolynomialBig)
        prod = p1*p2
        @assert leading(prod) == leading(p1)*leading(p2)
    end

    for _ in 1:N
        p_base = PolynomialBig(TermBig(1,0))
        for _ in 1:N_prods
            p = rand(PolynomialBig)
            prod = p_base*p
            @assert leading(prod) == leading(p_base)*leading(p)
            p_base = prod
        end
    end
    println("prod_test_polyBig - PASSED")
end

```
--- Polynomial product tests ---
prod_test_poly - PASSED
  0.471203 seconds (3.40 M allocations: 551.086 MiB, 27.62% gc time, 1.15% compilation time)
prod_test_polyBig - PASSED
 17.059532 seconds (85.48 M allocations: 3.471 GiB, 6.13% gc time, 0.11% compilation time)
```

I decreased the number of loops performed from the original code because it was taking too long to run; the desired results are clear even with these lower numbers. `PolynomialBig` is clearly far slower (over 34 times slower) than `Polynomial`, as we would expect from the larger memory allocation of `BigInt` compared to `Int64`.

## Task 5

## Task 6

## Task 7